In [1]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# import eli5

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

from numpy import array
from numpy import argmax
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Trial in sklearn data

In [5]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [6]:
# X, y = load_iris(return_X_y=True)
# clf = LogisticRegression(solver="liblinear").fit(X, y)
# roc_auc_score(y, clf.predict_proba(X), multi_class='ovr')

In [7]:
# X, y = load_iris(return_X_y=True)
# X, y = make_multilabel_classification(random_state=0)

# # clf = LogisticRegression(solver="liblinear").fit(X, y)
# clf_multi = MultiOutputClassifier(clf).fit(X, y)
# # get a list of n_output containing probability arrays of shape
# # (n_samples, n_classes)
# y_pred = clf_multi.predict_proba(X)
# # extract the positive columns for each output
# # y_pred = np.transpose([pred[:, 1] for pred in y_pred])
# # roc_auc_score(y, y_pred, average=None)

# Apply to real data

In [8]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\N.Babakov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
RST = 0
df_tr_test = pd.read_csv("../../Version2/sensitive_topics/compare_manual_manuallVSsemiauto/train_manualVSsemiauto_rst_{}.csv".format(RST))
df_val_test = pd.read_csv("../../Version2/sensitive_topics//compare_manual_manuallVSsemiauto/val_manual_only_rst_{}.csv".format(RST))
df_test_test = pd.read_csv("../../Version2/sensitive_topics/compare_manual_manuallVSsemiauto/test_manual_only_rst_{}.csv".format(RST))

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

In [ ]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=russian_stopwords)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])


In [ ]:
df_tr_test.columns[1:]

In [ ]:
for category in df_tr_test.columns[1:]:
    df_tr_test[category] = df_tr_test[category].apply(round)
    df_test_test[category] = df_tr_test[category].apply(round)
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(df_tr_test['text'], df_tr_test[category].tolist())
    # compute the testing accuracy
    prediction = NB_pipeline.predict(df_test_test['text'])
    prediction_prob = NB_pipeline.predict_proba(df_test_test['text'])[:,1]
    print('Test accuracy is {}'.format(accuracy_score(df_test_test[category].tolist(), prediction)))
    break

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support

In [14]:
def train_multi_topicbytopic(train ,test, pipeline):

    metrics_collected = []
    
    for category in train.columns[1:]:
        train[category] = train[category].apply(round)
        test[category] = test[category].apply(round)
        pipeline.fit(train['text'], train[category].tolist())
        
        prediction = pipeline.predict(test['text'])
        prediction_prob = pipeline.predict_proba(test['text'])[:,1]
        
        precision, recall, fscore, _ = precision_recall_fscore_support(test[category].tolist(), prediction, average = 'macro')
        
        rauc = roc_auc_score(test[category].tolist(),prediction_prob)
        
        metrics_collected.append([precision, recall, fscore,rauc])
    return np.mean(metrics_collected, axis = 0)
        
        
ppln = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=russian_stopwords)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

dt = train_multi_topicbytopic(df_tr_test,df_test_test, ppln)

C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.p

In [16]:
dt

array([0.60426485, 0.51956054, 0.51385374, 0.80119401])

In [17]:
len(df_tr_test),len(df_test_test)

(33100, 1585)

In [18]:
def get_NB_metric(pipeline):
    collected_results = []
    for rs in [0,1,2]:
        df_tr_curr = pd.read_csv("../../Version2/sensitive_topics/compare_manual_manuallVSsemiauto/train_manualVSsemiauto_rst_{}.csv".format(rs))
        df_test_curr = pd.read_csv("../../Version2/sensitive_topics/compare_manual_manuallVSsemiauto/test_manual_only_rst_{}.csv".format(rs))
        precision, recall, fscore,rocauc = train_multi_topicbytopic(df_tr_curr, df_test_curr,pipeline)
        collected_results.append([precision, recall, fscore, rocauc])
    return collected_results

ppln = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=russian_stopwords)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

nb_data = get_NB_metric(ppln)

C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.p

C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\N.Babakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.p

In [19]:
np.mean(nb_data,0)

array([0.59507056, 0.51810212, 0.51156595, 0.79124365])

In [27]:
np.std(nb_data,0)

array([0.01319604, 0.0013707 , 0.00233532, 0.00710139])

In [20]:
%%capture
ppln = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=russian_stopwords)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'))),
            ])

lreg_data = get_NB_metric(ppln)

In [21]:
np.mean(lreg_data,0)

array([0.90212897, 0.57228479, 0.59913501, 0.90590174])

In [28]:
np.std(lreg_data,0)

array([0.01879158, 0.00415966, 0.0056313 , 0.0065408 ])

In [24]:
from sklearn.linear_model import SGDClassifier

In [25]:
%%capture

ppln = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=russian_stopwords)),
                ('clf', OneVsRestClassifier(SGDClassifier(loss='log', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None))),
            ])

sgd_data = get_NB_metric(ppln)

In [26]:
np.mean(sgd_data,0)

array([0.46403441, 0.5       , 0.4810734 , 0.88044034])

In [29]:
np.std(sgd_data,0)

array([4.87447601e-06, 0.00000000e+00, 2.61607124e-06, 6.71540410e-03])